In [81]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

reading the csv

In [82]:
import pandas as pd

df = pd.read_csv("/content/depression_dataset_reddit_cleaned.csv")

print(df.head())

                                          clean_text  is_depression
0  we understand that most people who reply immed...              1
1  welcome to r depression s check in post a plac...              1
2  anyone else instead of sleeping more when depr...              1
3  i ve kind of stuffed around a lot in my life d...              1
4  sleep is my greatest and most comforting escap...              1


there was no seperate csv for testing and training so i used the same and split it later

In [83]:
train_df=pd.read_csv("/content/depression_dataset_reddit_cleaned.csv")
test_df=pd.read_csv("/content/depression_dataset_reddit_cleaned.csv")

length for each train and test

In [84]:
len(train_df)

7731

In [85]:
len(test_df)

7731

we shuffled the sample for getting better and accurate results

In [86]:
train_df_shuffled= train_df.sample(frac=1 , random_state=42)
train_df_shuffled.head()

,clean_text,is_depression
1199,my doctor think that sertaline is better for a...,1
3065,must be depression fr,1
2303,this ha ruined my life the only thing keeping ...,1
763,i am no job no career suffering from severe de...,1
4704,endearingevania i sure wish i could go with yo...,0


checked the name of columns

In [87]:
print(train_df_shuffled.columns)


Index(['clean_text', 'is_depression'], dtype='object')


#text and target in this case will be 'clean_text' and 'is_depression' and i checked it from csv excel file also

In [88]:
import random
random_index= random.randint(0, len(train_df)-1)
for row in train_df_shuffled[['clean_text','is_depression']][random_index: random_index+5].itertuples():
  index,text,target=row
  print(f"Target: {target};{'is a depression' if target!=0 else 'not a depression'}")
  print("Text:",text,end='\n')
  print("--"*20)

Target: 0;not a depression
Text: sleep for now then get up in hour then a very busy week aghhhhh
----------------------------------------
Target: 0;not a depression
Text: ha got a cold coming how shite
----------------------------------------
Target: 0;not a depression
Text: i m up so much paperwork to do today n i m kinda getting sick not fun
----------------------------------------
Target: 0;not a depression
Text: just found that if i use window to browse twitter com the firewall software will automatically turn off and the pc hang
----------------------------------------
Target: 1;is a depression
Text: tellyoursonthis it s called depression anxiety and stress in the west black magic someone want you harm while we don t consider other people s action and thought s to be evil or the reason for our misfortune
----------------------------------------


splitting it up into train and val for testing and training purposes

In [89]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences,train_labels, val_labels= train_test_split(train_df_shuffled['clean_text'].to_numpy(),
                                                                          train_df_shuffled['is_depression'].to_numpy(),
                                                                          test_size=0.2,
                                                                          random_state=42)

In [90]:
max_vocab_length=10000
max_length = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))
max_length

73

used text vectorization for vectorizing the sentence into parts for ML and better accuracy

In [91]:
text_vectorizer= tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                   output_sequence_length= max_length)

In [92]:
text_vectorizer.adapt(train_sentences)

In [93]:
random_sentence= random.choice(train_sentences)
print(random_sentence,'\n',len(random_sentence))
text_vectorizer([random_sentence])

nowadays i find that thing simply don t make me very happy like video game reading watching tv i have felt pretty neutral over the past couple of month but god i feel so stupid saying that the only thing that make my day feel like it ha meaning is my shit supermarket job where i rearrange product and basically ruin the day of whoever work in that aisle i just like being told to do something and getting away from my not so nice home life i m exhausted during every shift and have passed out twice at work before but it the only thing that make my life feel like it ha any point i feel like i want to take more hour so i can avoid this constant boredom but my job also make me feel so anxious and exhausted i don t know what to do it sound so pathetic and stupid 
 764


<tf.Tensor: shape=(1, 73), dtype=int64, numpy=
array([[3309,    2,  183,   12,   66,  832,   31,   10,   80,   11,  144,
         218,   25,  458,  399,  487,  522, 1192,    2,   18,  189,  295,
        3703,  114,    6,  242,  367,    9,  137,   15,  385,    2,   26,
          21,  346,  393,   12,    6,   90,   66,   12,   80,    7,   57,
          26,   25,    8,   74, 1032,   16,    7,  219, 3224,  128,  160,
           2,    1, 2133,    4,  564, 1252,    6,   57,    9, 1605,   85,
          13,   12,    1,    2,   20,   25,   91]])>

In [94]:
words=text_vectorizer.get_vocabulary()
words[:5]

['', '[UNK]', 'i', 'to', 'and']

In [95]:
embedding= tf.keras.layers.Embedding(input_dim= max_vocab_length,
                                     output_dim=128,
                                     input_length=max_length)

In [96]:
sample = embedding(text_vectorizer([random_sentence]))

In [97]:
sample[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.01405612,  0.01106758, -0.04494992,  0.03909068,  0.01040222,
       -0.01648103, -0.0217882 ,  0.02412069, -0.00259838,  0.03327436,
       -0.01452392, -0.00404637,  0.02443237, -0.02679579, -0.00190919,
        0.0111931 , -0.04544634,  0.02425113,  0.02450519, -0.00462966,
       -0.04260839, -0.00075669, -0.00441586,  0.0280008 , -0.0132751 ,
       -0.00077949, -0.00470157,  0.01527121, -0.00154557, -0.00012007,
       -0.02743926, -0.00441232,  0.00889683, -0.03324839, -0.04722452,
        0.03697543,  0.0006188 ,  0.0183001 ,  0.03077552,  0.03431368,
        0.04118579,  0.00109401, -0.02891039,  0.0478117 , -0.03903843,
        0.03282363, -0.00271   , -0.04873368,  0.03072191, -0.03468152,
        0.00138883, -0.02217691,  0.04356337, -0.04509835,  0.02270852,
       -0.03952408,  0.02615592, -0.01016507,  0.01527658, -0.01997445,
       -0.02973549,  0.03628162, -0.04811045, -0.01642662,  0.00875907,
       -0.048731

#BASE MODEL
making a base model to see the initial accuracy of the model

In [98]:
inputs= tf.keras.layers.Input(shape=(1,),dtype='string')
x= text_vectorizer(inputs)
x= embedding(x)
x= tf.keras.layers.GlobalAveragePooling1D()(x)
output= tf.keras.layers.Dense(1, activation='sigmoid')(x)
model= tf.keras.Model(inputs,output)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 73)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 73, 128)           1280000   
                                                                 
 global_average_pooling1d_1  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 1280129 (

In [99]:
history= model.fit(tf.expand_dims(train_sentences,axis=1),train_labels,
          epochs=5,
          validation_data=(val_sentences,val_labels))

Epoch 1/5
194/194 [==============================] - 14s 70ms/step - loss: 0.4643 - accuracy: 0.8372 - val_loss: 0.3399 - val_accuracy: 0.8778
Epoch 2/5
194/194 [==============================] - 5s 27ms/step - loss: 0.3187 - accuracy: 0.8858 - val_loss: 0.2982 - val_accuracy: 0.8901
Epoch 3/5
194/194 [==============================] - 4s 20ms/step - loss: 0.2776 - accuracy: 0.8994 - val_loss: 0.2662 - val_accuracy: 0.9037
Epoch 4/5
194/194 [==============================] - 3s 15ms/step - loss: 0.2374 - accuracy: 0.9156 - val_loss: 0.2319 - val_accuracy: 0.9198
Epoch 5/5
194/194 [==============================] - 2s 11ms/step - loss: 0.1985 - accuracy: 0.9293 - val_loss: 0.2030 - val_accuracy: 0.9328


#USING USE(UNIVERSAL SENTENCE ENCODER)
using (use) because previously i saw the dataset was pretty big so (use) being a efficient pre trained model will work better in this case

In [100]:
import tensorflow_hub as hub

embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])

print(embeddings)

tf.Tensor(
[[-0.03133019 -0.06338634 -0.01607501 ... -0.03242779 -0.0457574
   0.05370456]
 [ 0.05080859 -0.01652432  0.01573777 ...  0.00976659  0.03170119
   0.01788118]], shape=(2, 512), dtype=float32)


In [101]:
embed_sample= embed([random_sentence])
embed_sample

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[ 6.25393987e-02, -7.61995092e-02,  5.36653539e-03,
         5.69366552e-02,  5.55556938e-02,  4.33111377e-02,
         2.44599693e-02,  4.28485200e-02,  7.89523348e-02,
         5.13929836e-02, -3.23291235e-02,  2.46106163e-02,
         6.28240034e-03, -1.41590787e-02,  4.83526699e-02,
        -8.52187872e-02, -5.14160655e-02,  2.26961374e-02,
         6.00750558e-02,  3.33061405e-02,  4.33629565e-02,
        -2.62162159e-03,  7.51795396e-02,  1.18334619e-02,
        -4.77087405e-03,  8.90538748e-03, -5.31049930e-02,
         5.75728528e-02,  2.90661715e-02, -4.96279970e-02,
         2.58542150e-02,  2.02112477e-02,  4.84917052e-02,
        -1.37701100e-02,  1.17607033e-02,  4.70652096e-02,
         6.44179061e-02, -7.92928487e-02, -7.09431767e-02,
         2.31633950e-02, -4.67407741e-02,  5.00448719e-02,
        -3.60460840e-02, -3.42551246e-02,  6.02542311e-02,
         2.91299187e-02, -8.47179890e-02,  1.94373969e-02,
      

In [102]:
sentence_encoder= hub.KerasLayer(embed,
                                 input_shape=[],
                                 dtype=tf.string,
                                 trainable=False)

replacing the input shape and all with only one line (sentence encoder)

dropout i have used to ensure theres no overfitting

In [107]:
model1=tf.keras.models.Sequential([
    sentence_encoder,
    tf.keras.layers.Dense(64, activation='sigmoid'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')])
model1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
history= model1.fit(tf.expand_dims(train_sentences,axis=1),train_labels,
          epochs=5,
          validation_data=(val_sentences,val_labels))

Epoch 1/5
194/194 [==============================] - 12s 47ms/step - loss: 0.5418 - accuracy: 0.7317 - val_loss: 0.3179 - val_accuracy: 0.9502
Epoch 2/5
194/194 [==============================] - 8s 39ms/step - loss: 0.2838 - accuracy: 0.9080 - val_loss: 0.1806 - val_accuracy: 0.9593
Epoch 3/5
194/194 [==============================] - 8s 40ms/step - loss: 0.1883 - accuracy: 0.9398 - val_loss: 0.1314 - val_accuracy: 0.9625
Epoch 4/5
194/194 [==============================] - 9s 44ms/step - loss: 0.1425 - accuracy: 0.9541 - val_loss: 0.1103 - val_accuracy: 0.9644
Epoch 5/5
194/194 [==============================] - 10s 52ms/step - loss: 0.1219 - accuracy: 0.9594 - val_loss: 0.1004 - val_accuracy: 0.9670


this example was based on a real life situation where i asked a victim of depression to write me a paragraph of how he feels and the model predicted pretty well about the depression percentage so it can be used in real life scenarios

In [108]:
model1.predict(['will people be happy without me in their lives? Why do I need to blamed for literally everything I do. I am even tired of crying. It makes me feel sleepy. Why do I get on everyone nerves and torment them? I dont want to see my future anymore. It feels blur. I dont have the energy to imagine good things. Maybe if I wasnt born in this household they would have been happier.'])

1/1 [==============================] - 0s 444ms/step


array([[0.991629]], dtype=float32)

In [109]:
model1.predict(['depression'])

1/1 [==============================] - 0s 62ms/step


array([[0.996366]], dtype=float32)

In [110]:
loss, accuracy = model.evaluate(val_sentences, val_labels)
print("Accuracy:", accuracy)

49/49 [==============================] - 0s 3ms/step - loss: 0.2030 - accuracy: 0.9328
Accuracy: 0.9327731132507324
